# Movies recommender system :Stacked AutoEncoders

## Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


## Importing the dataset

In [ ]:
moviepath=r"path\ml-1m\movies.dat"
userpath=r"path\ml-1m\users.dat"
ratingpath=r"path\ml-1m\ratings.dat"
movies = pd.read_csv(moviepath, sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv(userpath, sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv(ratingpath, sep = '::', header = None, engine = 'python', encoding = 'latin-1')


## Preparing the training set and the test set

In [ ]:
trainpath=r"path\ml-100k\u1.base"
testpath=r"path\ml-100k\u1.test"
training_set = pd.read_csv(trainpath, delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv(testpath, delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


## Getting the number of users and movies

In [ ]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))


## Converting the data into an array with users in lines and movies in columns

In [ ]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)


## Converting the data into Torch tensors

In [ ]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


## Creating the architecture of the Neural Network

In [ ]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


## Training the SAE

In [ ]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 1.7651202128498944
epoch: 2 loss: 1.0965547060884973
epoch: 3 loss: 1.053346048910657
epoch: 4 loss: 1.0383756667892312
epoch: 5 loss: 1.030745903304824
epoch: 6 loss: 1.0265410746177421
epoch: 7 loss: 1.0235450251121503
epoch: 8 loss: 1.0218952797266574
epoch: 9 loss: 1.020714282884316
epoch: 10 loss: 1.0195700551415772
epoch: 11 loss: 1.0188920558897931
epoch: 12 loss: 1.0182781655431228
epoch: 13 loss: 1.0177885414143835
epoch: 14 loss: 1.017572395628434
epoch: 15 loss: 1.0172193223506945
epoch: 16 loss: 1.016908529402355
epoch: 17 loss: 1.0169711995987787
epoch: 18 loss: 1.0164122333158467
epoch: 19 loss: 1.0162197373422122
epoch: 20 loss: 1.0160201386374903
epoch: 21 loss: 1.0161780388269923
epoch: 22 loss: 1.0159066135265975
epoch: 23 loss: 1.0158699697290294
epoch: 24 loss: 1.015484227619337
epoch: 25 loss: 1.0157329698491815
epoch: 26 loss: 1.0156848109931396
epoch: 27 loss: 1.015410651486467
epoch: 28 loss: 1.0151204988229978
epoch: 29 loss: 1.012934373369784
ep

## Testing the SAE

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))


test loss: tensor(0.9476)
